# Chapter 7: Reflection

To reset your database, remove "store.db" and re-run ch04.ipynb & ch06.ipynb.

Necessary imports:

In [18]:
from sqlalchemy import MetaData, Table, create_engine, select, update

Create a metadata object to store schema information:

In [19]:
metadata = MetaData()

In [20]:
DATABASE_URL = "sqlite+pysqlite:///store.db"

Create the engine object:

In [21]:
engine = create_engine(
    DATABASE_URL,
)

Reflect the product table:

In [22]:
product = Table("product", metadata, autoload_with=engine)

In [23]:
print("Product table columns:", [c.name for c in product.columns])

Product table columns: ['product_id', 'product_name', 'unit_price', 'units_in_stock', 'type']


Note that the product table does not contain foreign keys, so only a single table is reflected:

In [24]:
print("Tables in metadata:", metadata.tables.keys())

Tables in metadata: dict_keys(['product'])


List products:

In [25]:
with engine.connect() as conn:
    stmt = (
        select(product)
        .order_by(product.c.product_id)
    )
    for row in conn.execute(stmt):
        print(row)

(1, 'phone', Decimal('300.00'), 4, 'PHONE')
(2, 'phone screen protector', Decimal('9.50'), 9, 'ACCESSORY')
(3, 'headphone', Decimal('25.99'), 9, 'ACCESSORY')
(4, 'digital camera', Decimal('45.99'), 5, 'OTHER')
(5, 'memory card 256GB', Decimal('21.99'), 0, 'ACCESSORY')


Reflecting the order table:

In [26]:
metadata = MetaData()
engine = create_engine(
    DATABASE_URL,
)

order = Table("order", metadata, autoload_with=engine)

Check which tables are loaded (note that multiple tables are loaded due to FK columns):

In [27]:
print("Tables loaded:", [key for key in metadata.tables.keys()])

Tables loaded: ['order', 'customer', 'employee']


In [28]:
print("Order table columns:", [c.name for c in order.columns])

Order table columns: ['order_id', 'customer_id', 'employee_id', 'order_datetime', 'is_shipped']


Employee table:

In [29]:
# employee `Table` object
employee = metadata.tables["employee"]

In [30]:
print("employee table columns:", [c.name for c in employee.columns])

employee table columns: ['employee_id', 'manager_id', 'name', 'is_manager', 'hire_date']


Assigning an employee to handle issues with an order:

In [31]:
with engine.connect() as conn:
    # find a suitable employee
    stmt1 = (
        select(employee.c.employee_id)
        .where(
            employee.c.is_manager == False,
            employee.c.manager_id != None,
        )
    )
    employee_id = conn.scalar(stmt1)
    print(f"Employee that should handle the issue: #{employee_id}")

    # find the problematic order (the 3rd order)
    # and assign it to the employee
    stmt2 = (
        select(order.c.order_id)
        .where(order.c.is_shipped == False)
    )
    order_id = conn.scalar(stmt2)
    print(f"The order with a problem: #{order_id}")

    # assign the order to the employee
    stmt3 = (
        update(order)
        .where(order.c.order_id == order_id)
        .values(employee_id=employee_id)
        .returning(order.c.order_id, order.c.employee_id)
    )
    updated_order = conn.execute(stmt3).first()
    print(
        f"Updated order: #{updated_order.order_id}, "
        f"assigned employee ID: #{updated_order.employee_id}."
    )

    conn.commit()

Employee that should handle the issue: #2
The order with a problem: #3
Updated order: #3, assigned employee ID: #2.


Reflecting All Tables:

In [32]:
metadata = MetaData()
engine = create_engine(
    DATABASE_URL,
)

metadata.reflect(bind=engine)

In [33]:
print("Tables in metadata:", metadata.tables.keys())

Tables in metadata: dict_keys(['customer', 'employee', 'order', 'order_detail', 'product'])


List products:

In [34]:
product = metadata.tables["product"]

with engine.connect() as conn:
    stmt = (
        select(product)
        .order_by(product.c.product_id)
    )
    for row in conn.execute(stmt):
        print(row)

(1, 'phone', Decimal('300.00'), 4, 'PHONE')
(2, 'phone screen protector', Decimal('9.50'), 9, 'ACCESSORY')
(3, 'headphone', Decimal('25.99'), 9, 'ACCESSORY')
(4, 'digital camera', Decimal('45.99'), 5, 'OTHER')
(5, 'memory card 256GB', Decimal('21.99'), 0, 'ACCESSORY')
